### **ЗАГРУЗКА БИБЛИОТЕК*

In [22]:
import pandas as pd
import numpy as np

# убираем предупреждения
import warnings
warnings.filterwarnings('ignore')

# парcинг
import requests
from bs4 import BeautifulSoup 

#  Регулярные выражения 
import re

# визуализация
import matplotlib.pyplot as plt
import seaborn as sns

# загрузка файлов с относительным путем
import glob
from pathlib import Path

from tkinter import filedialog
from tkinter import *

### **ПРОДАЖА СОПУТСТВУЮЩИХ ТОВАРОВ*

In [2]:
# Добавим кнопку выбора пути к папке
def browse_button():
    global path_sales
    filename = filedialog.askdirectory()
    path_sales.set(filename)
    print(filename)
root = Tk()
path_sales = StringVar()
lbl1 = Label(master=root,textvariable=path_sales)
lbl1.grid(row=0, column=1)
button2 = Button(text="Browse", command=browse_button)
button2.grid(row=0, column=3)

mainloop()

C:/Users/Anna/Desktop/ПРОЕКТ/ФАЙЛЫ_НАЦЕНКА


In [3]:
browse_button()

C:/Users/Anna/Desktop/ПРОЕКТ/ФАЙЛЫ_НАЦЕНКА


In [4]:
path_sales = input('Относительный путь к продажам - ')

# путь
p = Path(path_sales)

# находим xlsx
files = p.glob('*.xlsx')

# df
df_main = pd.concat([pd.read_excel(file, header=None, skiprows=2, skipfooter=1) for file in files])

Относительный путь к продажам - C:/Users/Anna/Desktop/ПРОЕКТ/ФАЙЛЫ_НАЦЕНКА


In [57]:
df_main.shape

(1640070, 10)

In [53]:
# Делаем копию объединения файлов - для быстрого возврата к исходнику 
sales = df_main.copy()

# Заполним отсутствующие данные по номеру чека, дате продажи и магазину предыдущими значениями
sales[[3, 4, 5]] = sales[[3, 4, 5]].fillna(method = 'ffill')

# Удалим строки, содержащие в столбце [0] возврат/чек/отчет и столбец с городом
sales = sales[~sales[0].str.contains('Чек|Отчет|Возврат')]
sales = sales.drop([2], axis=1)   
sales.head()
del1 = 1640070-sales[0].count()
balance = sales[0].count()
print('Чек/Возврат/Отчет')
print(f'удалено: {del1}')
print(f'остаток строк: {balance}')

Чек/Возврат/Отчет
удалено: 540726
остаток строк: 1099344


In [54]:
# В формате строки находится дата-время, преобразуем её в формат Timestamp и оставляем только дату
sales[4] = pd.to_datetime(sales[4], format="%d.%m.%Y %H:%M:%S").dt.date
sales[4] = pd.to_datetime(sales[4])

# Именуем колонки датафрейма
sales.columns =  ['product', 'group_of_products', 'receipt', 'date', 'store', 'sold_quantity', 
                  'purchase_cost_VAT', 'tunover_VAT', 'margin_VAT']

# Удаляем дубликаты 
sales = sales.drop_duplicates()
del2 = balance-sales.store.count()
balance = sales.store.count()
print('Дубликаты')
print(f'удалено: {del2}')
print(f'остаток строк: {balance}')

# Оставим в столбце store только наименование АЗС
def text_clean(store): 
   store = re.findall(r'\D{3}\s+\№\s?\d+', store)
   return store
sales.store = sales.store.apply(text_clean)
sales.store = [' '.join(text) for text in sales.store]

# Добавляем столбец city
sales['city'] = np.nan

# По АЗС №3 заполним город
sales.city.loc[sales.store == 'АЗС №3'] = 'Севастополь г'
sales.city.loc[sales.store == 'АЗС № 1'] = 'Симферополь г'
sales.city.loc[sales.store == 'АЗС №2'] = 'Керчь г'
sales.city.loc[sales.store == 'АЗС №4'] = 'Симферополь г'
sales.city.loc[sales.store == 'АЗС №11'] = 'Симферополь г'

# Удалим строки с группой "Составная Часть"
sales = sales.loc[~(sales.group_of_products == 'Составная Часть')]
del3 = del2-sales.store.count()
balance = sales.store.count()
print('Составная часть')
print(f'удалено: {del3}')
print(f'остаток строк: {balance}')


# Удалим строки, в которых одновременно нет данных по выручке и приобретению
sales = sales.loc[~((sales.purchase_cost_VAT.isna())&(sales.tunover_VAT.isna()))]
del4 = balance-sales.store.count()
balance = sales.store.count()
print('Отсутствуют выручка и стоимость приобретения')
print(f'удалено: {del4}')
print(f'остаток строк: {balance}')

# Заменим NaN в столбце tunover_VAT на 0
sales.tunover_VAT = sales.tunover_VAT.fillna(0)
sales.purchase_cost_VAT = sales.purchase_cost_VAT.fillna(0)
                    
# Рассчитаем значения столбцов
sales['margin_VAT'] = sales.tunover_VAT - sales.purchase_cost_VAT
sales['purchase_price_VAT'] = sales.purchase_cost_VAT / sales.sold_quantity
sales['profit'] = sales.margin_VAT / sales.sold_quantity

# Изменим порядок столбцов в датафрейме
sales = sales[['date', 'receipt', 'product', 'group_of_products', 'store', 'city', 'sold_quantity', 'tunover_VAT',
              'purchase_price_VAT', 'purchase_cost_VAT', 'margin_VAT', 'profit']]

Дубликаты
удалено: 384877
остаток строк: 714467
Составная часть
удалено: -328655
остаток строк: 713532
Отсутствуют выручка и стоимость приобретения
удалено: 15
остаток строк: 713517


In [7]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 713517 entries, 1 to 374482
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   date                713517 non-null  datetime64[ns]
 1   receipt             713517 non-null  float64       
 2   product             713517 non-null  object        
 3   group_of_products   713517 non-null  object        
 4   store               713517 non-null  object        
 5   city                713517 non-null  object        
 6   sold_quantity       713517 non-null  int64         
 7   tunover_VAT         713517 non-null  float64       
 8   purchase_price_VAT  713517 non-null  float64       
 9   purchase_cost_VAT   713517 non-null  float64       
 10  margin_VAT          713517 non-null  float64       
 11  profit              713517 non-null  float64       
dtypes: datetime64[ns](1), float64(6), int64(1), object(4)
memory usage: 70.8+ MB


In [8]:
# Создаем excel writer object 
out_path = 'C:/Users/Anna/Desktop/ПРОЕКТ/ФИНАЛ/sales_final.xlsx'
writer = pd.ExcelWriter(out_path , engine='xlsxwriter') 
# Записываем датафрейм в excel 
sales.to_excel(writer) 
# Сохраняем файл excel 
writer.save() 
print('Файл успешно выгружен')

Файл успешно выгружен


### **ПРОДАЖА ТОПЛИВА**

In [9]:
path_oil = input('Относительный путь к продажам - ')

# путь
o = Path(path_oil)

# находим xlsx
files_oil = o.glob('*.xlsx')

# df
df_oil = pd.concat([pd.read_excel(file, header=None, skiprows=6, skipfooter=1) for file in files_oil])

Относительный путь к продажам - C:/Users/Anna/Desktop/ПРОЕКТ/ФАЙЛЫ_ТОПЛИВО


In [13]:
# Делаем копию объединения файлов - для быстрого возврата к исходнику 
oil_sales = df_oil

# Заполняем отсутствующие номера чеков "1"
oil_sales[2] = oil_sales[2].fillna(1)

# Именуем колонки датафрейма
oil_sales.columns =  ['store', 'date', 'receipt', 'oil_product', 'quantity']

# Оставим в столбце store только наименование АЗС
def text_clean(store): 
   store = re.findall(r'\D{3}\s+\№\s?\d+', store)
   return store
oil_sales.store = oil_sales.store.apply(text_clean)
oil_sales.store = [' '.join(text) for text in oil_sales.store]

oil_sales.head()

,store,date,receipt,oil_product,quantity
0,АЗС № 1,20.09.2017 22:44:46,1.0,АИ-95-К5,2.353
1,АЗС № 1,20.09.2017 22:51:52,1.0,АИ-95-К5,2.000
2,АЗС № 1,20.09.2017 23:05:57,1.0,АИ-95-К5,2.000
3,АЗС № 1,20.09.2017 23:21:16,1.0,АИ-95-К5,2.000
4,АЗС № 1,20.09.2017 23:21:26,1.0,АИ-95-К5,2.824


In [14]:
oil_sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1604666 entries, 0 to 373844
Data columns (total 5 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   store        1604666 non-null  object 
 1   date         1604665 non-null  object 
 2   receipt      1604666 non-null  float64
 3   oil_product  1604666 non-null  object 
 4   quantity     1604666 non-null  float64
dtypes: float64(2), object(3)
memory usage: 73.5+ MB


In [15]:
# Создаем excel writer object 
out_path = 'C:/Users/Anna/Desktop/ПРОЕКТ/ФИНАЛ/oil_sales.xlsx'
writer = pd.ExcelWriter(out_path , engine='xlsxwriter') 
# Записываем датафрейм в excel 
sales.to_excel(writer) 
# Сохраняем файл excel 
writer.save() 
print('Файл успешно выгружен')

Файл успешно выгружен


### **ПОГОДА**

In [30]:
# Авторизуемся на сайте архива погоды
s = requests.Session()
data = {"login_username":"kav2005@bk.ru", "login_password":"19772004"}
url = "https://www.visualcrossing.com/account"

# Скачаем данные по погоде в городах: Симферополь, Севастополь, Керчь
with requests.Session() as s:
    simferopol = pd.read_csv('https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/simferopol/2017-01-01/2022-09-30?unitGroup=metric&include=days&key=NL2GLB33XY6PTWA6MH4J2HF97&contentType=csv',usecols = ['name', 'datetime', 'temp', 'precip', 'cloudcover', 'icon'])
    sevastopol = pd.read_csv('https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/sevastopol/2017-01-01/2022-09-30?unitGroup=metric&include=days&key=NL2GLB33XY6PTWA6MH4J2HF97&contentType=csv',usecols = ['name', 'datetime', 'temp', 'precip', 'cloudcover', 'icon'])
    kerch = pd.read_csv('https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/kerch/2017-01-01/2022-09-30?unitGroup=metric&include=days&key=NL2GLB33XY6PTWA6MH4J2HF97&contentType=csv',usecols = ['name', 'datetime', 'temp', 'precip', 'cloudcover', 'icon'])

HTTPError: HTTP Error 401: 

In [ ]:
# Изменим названия городов
simferopol['name'] = 'Симферополь г'
sevastopol['name'] = 'Севастополь г'
kerch['name'] = 'Керчь г'

# Соединим три датафрейма в один
weather = pd.concat((simferopol, sevastopol, kerch), axis=0)

# Переименуем столбцы datetime и name
weather.rename(columns={'name': 'city', 'datetime': 'date'}, inplace=True)

# Переведем столбец 'datetime' в тип данных datetime
weather['date'] = pd.to_datetime(weather['date'])
weather.info()

In [ ]:
# Посмотрим на уникальные значения в колонке 'icon'
print(weather['icon'].unique())

# Заменим описание погоды на русскоязычные значения
weather.icon.loc[(weather.icon == 'rain')] = 'Дождь'
weather.icon.loc[(weather.icon == 'clear-day')] = 'Ясно'
weather.icon.loc[(weather.icon == 'partly-cloudy-day')] = 'Переменная облачность'
weather.icon.loc[(weather.icon == 'snow')] = 'Снег'
weather.icon.loc[(weather.icon == 'cloudy')] = 'Облачно'
weather.icon.loc[(weather.icon == 'wind')] = 'Ветрено'
weather.head()

In [ ]:
# Создаем excel writer object 
out_path = 'C:/Users/Anna/Desktop/ПРОЕКТ/ФИНАЛ/weather.xlsx'
writer = pd.ExcelWriter(out_path , engine='xlsxwriter') 
# Записываем датафрейм в excel 
sales.to_excel(writer) 
# Сохраняем файл excel 
writer.save() 
print('Файл успешно выгружен')